In [ ]:
import os
import sys
import numpy as np
import random
import time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope

# Load Inception-v3 graph from slim
with slim.arg_scope(inception_v3_arg_scope()):
    x = tf.placeholder(tf.float32, shape=[None, None, None, 3], name='input_im')
    inception_v3(x, num_classes=1001, is_training=False)

# Write graph layout to file
tf.summary.FileWriter('./slim_graph/', graph=tf.get_default_graph())
print('Graph layout saved to {}'.format('./slim_graph/'))

In [1]:
import os
import sys
import numpy as np
import random
import time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope

branch_path='InceptionV3/InceptionV3/Mixed_6d/concat:0'
layers_to_train=['Mixed_6d', 'Mixed_6c', 'Mixed_6b']
own_layers=3
init_learning_rate=1e-3
lr_decay_freq=1
lr_decay_factor=1,
epsilon=0.01
alpha=0.95
activation='tanh'

# Define preprocessing
with tf.name_scope('Preprocessing'):
    input_im = tf.placeholder(tf.uint8, shape=[None, None, None, 3], name='input_im')
    resized_im = tf.image.resize_bilinear(tf.image.convert_image_dtype(
        tf.convert_to_tensor(input_im), dtype=tf.float32), [299, 299], name='resized_im')
    normalized_im = tf.multiply(tf.subtract(resized_im, 0.5), 2.0, name='normalized_im')

# Load Inception-v3 graph from slim
with slim.arg_scope(inception_v3_arg_scope()):
    inception_v3(normalized_im, num_classes=1001, is_training=False)
    
# Create own branch
with tf.name_scope('Own'):
    y = tf.placeholder(dtype="float", shape=[None, 300], name='y')

    # Branch from original network and shape tensor
    x = tf.get_default_graph().get_tensor_by_name(branch_path)
    branch_size = int(x.get_shape()[-1])
    x = tf.nn.avg_pool(x, ksize=[1]+list(x.get_shape()[1:3])+[1], strides=[1, 1, 1, 1], padding='VALID')
    x = tf.squeeze(x, [1, 2])

    # Calculate dimensions between own layers
    p = [int(300 + i * (branch_size - 300) / own_layers) for i in range(own_layers + 1)]
    p = p[::-1]

    # Build own fully-connected layers
    w, b, a, n, z = [], [], [], [], []
    training = tf.placeholder(tf.bool, name='phase')
    for i in range(own_layers):
        w.append(tf.Variable(tf.random_normal([p[i], p[i+1]], stddev=float('1e-5')), name='w_'+str(i)))
        b.append(tf.Variable(tf.random_normal([1, p[i+1]]), name='b_'+str(i)))
        if i == 0:
            a.append(tf.add(tf.matmul(x, w[i]), b[i], name='a_'+str(i)))
        else:
            a.append(tf.add(tf.matmul(z[i-1], w[i]), b[i], name='a_'+str(i)))
        if i+1 != own_layers:
            n.append(tf.layers.batch_normalization(a[i], training=training, name='n_'+str(i)))
            if activation == 'relu':
                z.append(tf.nn.relu(n[i], name='relu_' + str(i)))
            elif activation == 'tanh':
                z.append(tf.nn.tanh(n[i], name='tanh_' + str(i)))
            else:
                print("'" + activation + "' is not a valid activation function. Falling back to 'tanh'.")
                z.append(tf.nn.tanh(n[i], name='tanh_' + str(i)))

    y_pred = tf.identity(a[-1], name='y_pred')
    with tf.name_scope('L2_distance'):
        l2_dist = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(y - y_pred), axis=1)), name='l2_dist')
        tf.summary.scalar(name='l2_distance', tensor=l2_dist)
    with tf.name_scope('Cosine_distance'):
        cos_dist = tf.losses.cosine_distance(tf.nn.l2_normalize(y, dim=1),
                                             tf.nn.l2_normalize(y_pred, dim=1), dim=1)
        tf.summary.scalar(name='cosine_distance', tensor=cos_dist)
    with tf.name_scope('Mixed_distance'):
        mixed_dist = tf.add(alpha*cos_dist, (1-alpha)*l2_dist, name='mixed_dist')
        tf.summary.scalar(name='mixed_distance', tensor=mixed_dist)
    with tf.name_scope('Misc_summaries'):
        y_pred_norm = tf.reduce_mean(tf.norm(y_pred, axis=1), name='y_pred_norm')
        tf.summary.scalar(name='pred_norm', tensor=y_pred_norm)
        norm_diff = tf.reduce_mean(tf.norm(y_pred, axis=1) - tf.norm(y, axis=1))
        tf.summary.scalar(name='norm_diff', tensor=norm_diff)
        mean_tiled = tf.tile(tf.expand_dims(tf.reduce_mean(y, axis=0), 0), tf.stack([tf.shape(y_pred)[0], 1]))
        mean_diff = tf.losses.cosine_distance(tf.nn.l2_normalize(mean_tiled, dim=1),
                                             tf.nn.l2_normalize(y_pred, dim=1), dim=1)
        tf.summary.scalar(name='mean_diff', tensor=mean_diff)

# Define variables to train
if layers_to_train == ['all']:
    variables_to_train = tf.trainable_variables()
else:
    variables_to_train = w + b
    for layer in layers_to_train:
        variables_to_train.extend([v for v in tf.global_variables() if
                                   v.name.startswith('InceptionV3/'+layer) and
                                   (v.name.endswith('weights:0') or
                                    v.name.endswith('biases:0') or
                                    v.name.endswith('beta:0'))])

# Define optimization
with tf.name_scope('Optimization'):
    global_step = tf.Variable(0, trainable=False, name='global_step')
    learning_rate = tf.train.exponential_decay(init_learning_rate, global_step=global_step,
                                               decay_steps=lr_decay_freq, decay_rate=lr_decay_factor,
                                               name='learning_rate')
    adam = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=epsilon, name='Adam')
    train_step = adam.minimize(mixed_dist, var_list=variables_to_train, global_step=global_step, name='train_step')
    
# Write graph layout to file
tf.summary.FileWriter('./slim_graph/', graph=tf.get_default_graph())
print('Graph layout saved to {}'.format('./slim_graph/'))

ValueError: Shape must be rank 0 but is rank 1 for 'Optimization/train_step/update_Own/w_0/ApplyAdam' (op: 'ApplyAdam') with input shapes: [768,612], [768,612], [768,612], [], [], [1], [], [], [], [768,612].